# Foundations of Computer Science Project
## University dataset

**Martino Lorenzo Tenconi 803154**

**1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes**

**2. For each university, compute the improvement in income between the least recent and the most recent data points**

**3. Find the university with the largest increase computed in the previous point**

**4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range**

**5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.**

**6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)**

**7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).**

**8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name**

**9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value**

**10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).**

**11. In the times ranking, compute the number of times each university appears**

**12. Find the universities that appear at most twice in the times ranking.**

**13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).**

Import dei pacchetti necessari 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import time
import re
from collections import Counter

**1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes**

Esplorazione del dataset _times_

In [2]:
times_data = pd.read_csv("data/timesData.csv")
times_data

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,"21,958",15.3,3%,48 : 52,2016
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,"31,268",28.7,2%,36 : 64,2016
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,"4,122",3.7,3%,NaN,2016
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8%,28 : 72,2016


Si osserva che i valori possibili per il campo _year_ sono tutti gli anni dal 2011 al 2016. I valori più recenti sono quindi quelli riferiti al massimo all'anno 2016, mentre i meno recenti al minimo al 2011.

In [3]:
years = np.unique(times_data['year'])
years

array([2011, 2012, 2013, 2014, 2015, 2016])

Si controlla se ci sono righe che presentano valore nullo per il campo _year_. Nessuna riga ha valore nullo per questa colonna.

In [4]:
nan_years = times_data[times_data['year'].isnull()]
nan_years

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year


Poichè non ci sono valori nulli nella colonna *year*, possono essere creati due dataframe, rispettivamente con i dati più recenti di ciascuna università e per i dati più vecchi.

In [5]:
recent_times_data = times_data.loc[times_data.groupby(["university_name"])["year"].idxmax()].reset_index(drop=True)

In [6]:
recent_times_data

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
1,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
3,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
814,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
815,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
816,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


In [7]:
old_times_data = times_data.loc[times_data.groupby(["university_name"])["year"].idxmin()].reset_index(drop=True)

In [8]:
old_times_data

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011
814,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011
815,39,École Polytechnique,France,57.9,77.9,56.1,91.4,-,69.5,"2,429",4.8,30%,18 : 82,2011
816,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011


Si nota che per ogni università si possono avere diversi livelli di aggiornamento del dato: per alcune università il dato più recente è relativo al 2016, per altre può esserlo di un anno più lontano nel tempo. 

**2. For each university, compute the improvement in income between the least recent and the most recent data points**

Per questo task la strategia è di effettuare un merge dei due dataframe appena creati *recent_times_data* e *old_times_data* sulla colonna *university_name*. Prima di calcolare la differenza in income tra il dato più recente e quello meno recente, è necessario convertire il tipo della colonna stessa da stringa a valore numerico. A questo punto si può facilmente calcolare la tabella risultato con la differenza in income per ciascuna università.
NB: Quando un'università ha un valore nullo per la colonna *income*, la differenza tra un numero e un valore nullo è stata considerata come valore nullo, poichè non si sa attribure un valore al valore nullo.

In [9]:
recent_times_data = recent_times_data.replace('-', value = np.nan)
recent_times_data['income'] = recent_times_data.income.astype(float)
old_times_data = old_times_data.replace('-', value = np.nan)
old_times_data['income'] = old_times_data.income.astype(float)

In [10]:
merged_df = recent_times_data.merge(old_times_data, on='university_name')

In [11]:
merged_df['income_diff'] = merged_df['income_x'] - merged_df['income_y']

In [12]:
income_diff_data = merged_df[['university_name','income_diff']]

In [13]:
income_diff_data

,university_name,income_diff
0,AGH University of Science and Technology,NaN
1,Aalborg University,7.3
2,Aalto University,-0.3
3,Aarhus University,6.8
4,Aberystwyth University,-4.2
...,...,...
813,École Normale Supérieure,6.4
814,École Normale Supérieure de Lyon,5.6
815,École Polytechnique,NaN
816,École Polytechnique Fédérale de Lausanne,27.4


Si ordinano i valori per il valore di *income_diff* e si estraggono le università rispettivamente con maggiore e minore differenza in income.

In [14]:
not_null_income_diff = income_diff_data[~income_diff_data['income_diff'].isnull()].sort_values('income_diff',
                                                                                              ascending = False)

In [15]:
not_null_income_diff.head()

,university_name,income_diff
428,TU Dresden,67.8
278,Nanyang Technological University,59.9
230,LMU Munich,59.6
205,Karlsruhe Institute of Technology,59.5
276,Nagoya University,58.3


L'università di Dresda è l'università con la maggior differenza in income.

In [16]:
not_null_income_diff.tail()

,university_name,income_diff
435,Technical University of Denmark,-45.5
587,University of Iceland,-47.4
237,Leiden University,-50.2
491,University of Arizona,-51.8
414,State University of New York Albany,-64.9


L'università di Albany dello stato di New York è quella con la maggior differenza negativa in income.

**3. Find the university with the largest increase computed in the previous point**

In [17]:
index_max_diff_income = income_diff_data['income_diff'].idxmax()

In [18]:
income_diff_data.iloc[index_max_diff_income]

university_name    TU Dresden
income_diff              67.8
Name: 428, dtype: object

**4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range**

Si caricano i dataset *cwur* e *shanghai*.

In [19]:
cwur_data = pd.read_csv("data/cwurData.csv")

In [20]:
shanghai_data = pd.read_csv("data/shanghaiData.csv")

Per entrambi i dataset appena caricati, si calcola, come fatto per il dataset *times*, il dataset con i dati più recenti per ciascuna università.

In [21]:
recent_cwur_data = cwur_data.iloc[cwur_data.groupby(['institution'])['year'].idxmax()].reset_index(drop=True)

In [22]:
recent_shangai_data = shanghai_data.iloc[shanghai_data.groupby(['university_name'])['year'].idxmax()].reset_index(drop=True)

Per calcolare la differenza massima tra i ranking si uniscono i 3 dataset con i dati più recenti per ogni ranking. Per fare questo si effettua prima un merge tra il dataset *recent_times_data* e *recent_cwur_data* e successivamente un secondo merge tra il dataset appena ottenuto e *recent_shanghai_data*.

In [23]:
df1 = recent_times_data.merge(recent_cwur_data, left_on = 'university_name', right_on = 'institution', suffixes = ('_times', '_cwur'))

In [24]:
all_recent_data = df1.merge(recent_shangai_data, on = 'university_name')

Poichè la colonna *world_rank* del dataset *times* ha un simbolo di '=' a precedere la posizione dell'università nel ranking, si elimina questo simbolo che non ha significato e anzi potrebbe risultare scomodo per gli scopi di questo task.

In [25]:
all_recent_data['world_rank_times'] = all_recent_data['world_rank_times'].str.replace('=', '')

Si estrae una porzione del dataset ottenuto, in modo che contenga le sole colonne di interesse, ovvero i 3 ranking per ciascuna università: *world_rank_times*, *world_rank_cwur*, *world_rank*

In [26]:
ranking_data = all_recent_data[['university_name', 'world_rank_times', 'world_rank_cwur', 'world_rank']].reset_index(drop=True)

In [27]:
ranking_data.head()

,university_name,world_rank_times,world_rank_cwur,world_rank
0,Aalborg University,201-250,565,301-400
1,Aalto University,251-300,421,401-500
2,Aarhus University,106,122,73
3,Aix-Marseille University,251-300,206,102-150
4,Aristotle University of Thessaloniki,601-800,459,401-500


Si inizializza una funzione *convert_to_int* che prende in input un elemento, che può essere di tipo stringa o già di tipo intero. Se l'elemento in input è già di tipo intero, allora la funzione si limita a restituire una lista contenente come unico elemento il numero stesso. Altrimenti, se l'elemento è di tipo stringa ma rappresenta un intero (es '49'), prova a convertirtlo subito in intero e creare quindi la lista lunga 1 come spiegato in precedenza. Se invece l'elemento è una stringa del tipo '301-400', la funzione resituisce una lista contenente i due valori separati da '-', convertiti a tipo numerico.

In [28]:
def convert_to_int(elem):
    if isinstance(elem, str):
        try:
            num = [int(elem)]
        except:
            num = [int(x) for x in elem.split('-')]
    elif isinstance(elem, int):
        num = [elem]
    return num

La funzione *ranking_diff* prende in input una riga del dataset e calcola il minimo e il massimo dei valori del ranking per quella riga. Infine ritorna la differenza in ranking come differenza tra il valore massimo e il valore minimo.

In [29]:
def ranking_diff(row):
    #rank1 = convert_to_int(row['world_rank_times'])
    #rank2 = convert_to_int(row['world_rank_cwur'])
    #rank3 = convert_to_int(row['world_rank'])
    
    r1 = row['world_rank_times']
    r2 = row['world_rank_cwur']
    r3 = row['world_rank']
    
    rank1 = convert_to_int(r1)
    rank2 = convert_to_int(r2)
    rank3 = convert_to_int(r3)
    
    
    rankings = [rank1, rank2, rank3]
    min_values = [np.min(values) for values in rankings]
    max_values = [np.max(values) for values in rankings]
    
    min_value = np.min(min_values)
    max_value = np.max(max_values)
    
    rank_diff = max_value - min_value
    
    return rank_diff

Si applica la funzione *ranking_diff* al dataset *ranking_data* e si salva il risultato della differenza in una nuova colonna.

In [30]:
ranking_data

,university_name,world_rank_times,world_rank_cwur,world_rank
0,Aalborg University,201-250,565,301-400
1,Aalto University,251-300,421,401-500
2,Aarhus University,106,122,73
3,Aix-Marseille University,251-300,206,102-150
4,Aristotle University of Thessaloniki,601-800,459,401-500
...,...,...,...,...
357,Yale University,12,11,11
358,Yeshiva University,164,171,201-300
359,Yonsei University,301-350,98,201-300
360,York University,301-350,337,401-500


In [31]:
ranking_data.loc[:,'maximum_rank_diff'] = ranking_data.apply(ranking_diff, axis = 1)

In [32]:
ranking_data

,university_name,world_rank_times,world_rank_cwur,world_rank,maximum_rank_diff
0,Aalborg University,201-250,565,301-400,364
1,Aalto University,251-300,421,401-500,249
2,Aarhus University,106,122,73,49
3,Aix-Marseille University,251-300,206,102-150,198
4,Aristotle University of Thessaloniki,601-800,459,401-500,399
...,...,...,...,...,...
357,Yale University,12,11,11,1
358,Yeshiva University,164,171,201-300,136
359,Yonsei University,301-350,98,201-300,252
360,York University,301-350,337,401-500,199


Si nota che come indicato nella consegna del punto 4., la *Aarhus University* ha differenza dei ranking uguale a 49.

In [33]:
sorted_ranking_data = ranking_data.sort_values('maximum_rank_diff',ascending = False)

In [34]:
sorted_ranking_data.head()

,university_name,world_rank_times,world_rank_cwur,world_rank,maximum_rank_diff
88,King Abdulaziz University,251-300,995,151-200,844
353,Waseda University,601-800,38,401-500,762
321,University of Tehran,601-800,768,201-300,599
80,Jilin University,601-800,391,201-300,599
86,Keio University,501-600,34,301-400,566


In [35]:
sorted_ranking_data.tail()

,university_name,world_rank_times,world_rank_cwur,world_rank,maximum_rank_diff
299,University of Pennsylvania,17,14,17,3
229,University of Chicago,10,8,9,2
171,Stanford University,3,2,2,1
357,Yale University,12,11,11,1
226,University of Cambridge,4,4,5,1


**5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.**

Innanzitutto si verifica se la colonna *female_male_ratio* contiene valori nulli.

In [36]:
recent_times_data[recent_times_data['female_male_ratio'].isnull()]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,"35,569",17.0,1%,NaN,2016
15,601-800,Amrita University,India,20.4,29.6,5.8,20.7,28.0,NaN,"17,273",6.6,8%,NaN,2016
21,401-500,Aston University,Unted Kingdom,18.4,92.5,20.4,51.9,34.6,NaN,NaN,NaN,NaN,NaN,2016
34,601-800,Beijing Institute of Technology,China,25.9,16.9,21.7,9.1,87.6,NaN,NaN,NaN,NaN,NaN,2016
38,351-400,Bilkent University,Turkey,22.6,47.5,17.5,64.1,30.2,NaN,NaN,NaN,NaN,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,=131,University of York,United Kingdom,41.9,83.5,46.3,73.1,31.1,55.4,NaN,NaN,NaN,NaN,2016
793,=60,Washington University in St Louis,United States of America,54.3,44.9,55.6,99.2,33.5,67.0,"12,528",5.7,17%,NaN,2016
798,401-500,Wuhan University,China,38.2,36.7,26.8,18.0,97.8,NaN,"46,227",14.4,7%,NaN,2016
806,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,NaN,"4,122",3.7,3%,NaN,2016


Si eliminano dal dataset le osservazioni relative a queste 79 università.

In [37]:
female_male_data = recent_times_data[~recent_times_data['female_male_ratio'].isnull()].reset_index(drop=True)

Si inizializza la funzione *female_count*, che prende in input una riga di un dataset e calcola il rapporto tra il numero di donne e di uomini tra gli iscritti in università. Il numero di donne è appunto derivato dal numero totale di studenti moltiplicato dal rapporto tra il numero di donne e di uomini.

In [38]:
def female_count(row):
    ratio = row['female_male_ratio']
    total_number = row['num_students']
    total_number = int(total_number.replace(',',''))
    perc_female = int(ratio.split(':')[0])
    perc_male = int(ratio.split(':')[1])
    n_female = round(total_number * perc_female/100)
    
    return n_female

La funzione *male_count* verrà applicata a ciascuna riga dopo l'applicazione della funzione *female_count* per restituire semplicemente il numero di uomini come numero totale di studenti - numero di donne.

In [39]:
def male_count(row):
    total_number = row['num_students']
    total_number = int(total_number.replace(',',''))
    female_number = row['num_female']
    n_male = total_number - female_number
    return n_male

In [40]:
female_male_data.loc[:,'num_female'] = female_male_data.apply(female_count, axis = 1)

In [41]:
female_male_data.loc[:,'num_male'] = female_male_data.apply(male_count, axis = 1)

Si raggruppa quindi per paese e si somma il numero di uomini e di donne iscritti all'università per ciascuna nazione.

In [42]:
female_male_num_data = female_male_data.groupby(['country'])[['num_female','num_male']].sum()

In [43]:
female_male_num_data

,num_female,num_male
country,,
Argentina,67191,41182
Australia,391736,321640
Austria,68364,66113
Bangladesh,21323,41393
Belarus,20219,9084
...,...,...
Uganda,18670,18670
Ukraine,17846,19250
United Arab Emirates,9516,4931


**6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)**

La funzione *compute_ratio* prende in input una riga del dataset e calcola il numero di donne iscritte all'università di riferimento di quella stessa riga per ogni uomo iscritto. Se ad esempio il rapporto è 54% donne e 46% uomini, il valore del rapporto sarà 54/46 = 1.17.

In [44]:
def compute_ratio(row):
    ratio = row['female_male_ratio']
    female = int(ratio.split(':')[0])
    male = 100 - female
    if male != 0:
        female_ratio = female/male
    else:
        female_ratio = 100
    return female_ratio

Si applica questa funzione a tutte le righe del dataset *female_male_data*

In [45]:
female_male_data.loc[:,'female_ratio'] = female_male_data.apply(compute_ratio, axis = 1)

Si calcola il valore medio di tale rapporto su tutte le righe del dataset e si estraggono dal dataset tutte e sole le università che hanno un rapporto donne-uomini inferiore a quello medio.

In [46]:
mean_female_ratio = female_male_data['female_ratio'].mean()

In [47]:
mean_female_ratio

1.2149571636836476

In [48]:
sub_female_data = female_male_data[female_male_data['female_ratio'] < mean_female_ratio].reset_index(drop=True)

In [49]:
sub_ratio_female = sub_female_data[['university_name','country', 'female_ratio']]

In [50]:
sub_ratio_female

,university_name,country,female_ratio
0,Aalborg University,Denmark,0.923077
1,Aalto University,Finland,0.470588
2,Aarhus University,Denmark,1.173913
3,Aberystwyth University,United Kingdom,0.923077
4,Ajou University,South Korea,0.492537
...,...,...,...
463,Zhejiang University,China,0.694915
464,École Normale Supérieure,France,0.851852
465,École Normale Supérieure de Lyon,France,0.960784
466,École Polytechnique,France,0.219512


In [51]:
sorted_sub_ratio_female = sub_ratio_female.sort_values('female_ratio', ascending = False)

In [52]:
sorted_sub_ratio_female.head()

,university_name,country,female_ratio
79,George Mason University,United States of America,1.173913
440,Université Libre de Bruxelles,Belgium,1.173913
452,William & Mary,United States of America,1.173913
378,University of Modena and Reggio Emilia,Italy,1.173913
220,Queensland University of Technology,Australia,1.173913


In [53]:
sorted_sub_ratio_female.tail()

,university_name,country,female_ratio
277,Tokyo Institute of Technology,Japan,0.149425
206,Polytechnic University of Catalonia,Spain,0.136364
130,Kyushu Institute of Technology,Japan,0.123596
286,Toyohashi University of Technology,Japan,0.098901
210,Pontifical Catholic University of Paraná,Brazil,0.010101


**7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).**

Si calcola quindi il numero totale di studenti per ogni nazione e successivamente per le sole università con rapporto donne:uomini inferiore alla media.

In [54]:
female_male_num_data['total_students'] = female_male_num_data['num_female'] + female_male_num_data['num_male']

In [55]:
female_male_num_data

,num_female,num_male,total_students
country,,,
Argentina,67191,41182,108373
Australia,391736,321640,713376
Austria,68364,66113,134477
Bangladesh,21323,41393,62716
Belarus,20219,9084,29303
...,...,...,...
Uganda,18670,18670,37340
Ukraine,17846,19250,37096
United Arab Emirates,9516,4931,14447


In [56]:
total_sub_students = sub_female_data['num_female'] + sub_female_data['num_male']

selected_sub_female_data = sub_female_data[['university_name','country']]

In [57]:
selected_sub_female_data.loc[:,'total_students'] = total_sub_students

/usr/lib/python3/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib/python3/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Si raggruppano le righe del dataset *selected_sub_female_data* per paese e si effettua una somma del numero di studenti.

In [58]:
country_total_students_sub_female = selected_sub_female_data.groupby('country')['total_students'].sum().reset_index()

Si uniscono le righe dei due dataset con un merge sul nome della nazione e si calcola dunque il rapporto tra il numero di studenti presenti in università che hanno rapporto donne:uomini iscritti inferiore alla media e studenti totali in quella nazione.

In [59]:
country_data = country_total_students_sub_female.merge(female_male_num_data, on = 'country')

country_data['students_ratio'] = country_data['total_students_x']/country_data['total_students_y']

In [60]:
country_data

,country,total_students_x,num_female,num_male,total_students_y,students_ratio
0,Australia,295021,391736,321640,713376,0.413556
1,Austria,79242,68364,66113,134477,0.589261
2,Bangladesh,62716,21323,41393,62716,1.000000
3,Belgium,116129,92920,76741,169661,0.684477
4,Brazil,438476,247271,246980,494251,0.887152
5,Canada,229920,344440,279189,623629,0.368681
6,Chile,90951,37962,52989,90951,1.000000
7,China,918214,427842,587308,1015150,0.904511
8,Colombia,54645,26376,28269,54645,1.000000
9,Czech Republic,54324,97971,77900,175871,0.308885


In [61]:
sorted_country_data = country_data.sort_values('students_ratio', ascending = False)

Si mostrano i 5 paesi che hanno un maggior numero di studenti concentrato in università con tasso femminile basso rispetto alla media. In particolare, tutti gli studenti di questi paesi si concentrano in università che hanno un una forte prevalenza di uomini rispetto alle donne iscritte.

In [62]:
sorted_country_data.head()

,country,total_students_x,num_female,num_male,total_students_y,students_ratio
28,Luxembourg,5144,2572,2572,5144,1.0
31,Morocco,56060,25788,30272,56060,1.0
16,Ghana,37274,15282,21992,37274,1.0
36,Oman,7089,3544,3545,7089,1.0
37,Pakistan,22496,9373,13123,22496,1.0


Questi invece i paesi che hanno un minor numero di studenti concentrato in università con tasso femminile basso rispetto alla media. 

In [63]:
sorted_country_data.tail()

,country,total_students_x,num_female,num_male,total_students_y,students_ratio
38,Poland,59789,133460,91853,225313,0.265360
19,Hungary,22447,45546,42219,87765,0.255763
24,Italy,206915,642678,486489,1129167,0.183246
44,South Africa,41889,202378,127820,330198,0.126860
29,Malaysia,16270,99276,62551,161827,0.100539


**8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name**

In [64]:
educational_data = pd.read_csv("data/educational_attainment_supplementary_data.csv")

In [65]:
educational_data

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
educational_data_not_null = educational_data[educational_data['series_name'].notna() & educational_data['country_name'].notna()].reset_index()

In [67]:
educational_data_not_null

,index,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


**9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value**

Si visualizzano le colonne presenti nel dataset *educational_attainment*.

In [68]:
educational_data_not_null.columns

Index(['index', 'country_name', 'series_name', '1985', '1986', '1987', '1990',
       '1991', '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2015'],
      dtype='object')

L'operazione effettuata è un *melt*: per ogni riga duplicare l'informazione di *country_name* e *series_name*, una volta ciascuno per ogni anno, a cui corrisponderà un valore.

In [69]:
melted_educational_data = pd.melt(educational_data_not_null, id_vars = ['country_name','series_name'], var_name = 'year')

In [70]:
melted_educational_data

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",index,0.0
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",index,1.0
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",index,2.0
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",index,3.0
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",index,4.0
...,...,...,...,...
2213395,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2213396,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2213397,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2213398,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


**10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).**

La strategia scelta è quella di estrarre i valori unici dei nomi per ciascuno dei 3 rankings e poi concatenare queste 3 liste in un unico grande elenco. Attraverso la funzione Counter dela libreria *collections*, si contano poi le occorrenze per ciascuna università e si costruisce un dataframe con queste informazioni.

In [71]:
times_universities = list(pd.unique(times_data['university_name']))

In [72]:
shanghai_universities = list(pd.unique(shanghai_data['university_name']))

In [73]:
cwur_universities = list(pd.unique(cwur_data['institution']))

In [74]:
all_universities = times_universities + shanghai_universities + cwur_universities

In [75]:
count_universities = dict(Counter(all_universities))

In [76]:
n_rankings_universities = pd.DataFrame(count_universities.items(), columns=['university_name', 'n_rankings'])

In [77]:
n_rankings_universities

,university_name,n_rankings
0,Harvard University,3
1,California Institute of Technology,3
2,Massachusetts Institute of Technology,2
3,Stanford University,3
4,Princeton University,3
...,...,...
1448,Shenzhen University,1
1449,Tianjin Medical University,1
1450,Babeș-Bolyai University,1
1451,Henan Normal University,1


Alcune delle università che compaiono in tutti e tre i rankings sono:

In [78]:
n_rankings_universities.head()

,university_name,n_rankings
0,Harvard University,3
1,California Institute of Technology,3
2,Massachusetts Institute of Technology,2
3,Stanford University,3
4,Princeton University,3


Alcune delle università che compaiono solo in un ranking:

In [79]:
n_rankings_universities.tail()

,university_name,n_rankings
1448,Shenzhen University,1
1449,Tianjin Medical University,1
1450,Babeș-Bolyai University,1
1451,Henan Normal University,1
1452,Southwest Jiaotong University,1


**11. In the times ranking, compute the number of times each university appears**

Si utilizza la stessa strategia, ma solo sui nomi che compaiono nel dataset *times_data*.

In [80]:
universities_times = list(times_data['university_name'])

In [81]:
n_times_universities = Counter(universities_times)

In [82]:
n_times_universities_data = pd.DataFrame(n_times_universities.items(), columns=['university_name', 'appereances'])

In [83]:
n_times_universities_data

,university_name,appereances
0,Harvard University,6
1,California Institute of Technology,6
2,Massachusetts Institute of Technology,6
3,Stanford University,6
4,Princeton University,6
...,...,...
813,Xidian University,1
814,Yeungnam University,1
815,Yıldız Technical University,1
816,Yokohama City University,1


**12. Find the universities that appear at most twice in the times ranking.**

In [84]:
not_frequent_universities = n_times_universities_data[n_times_universities_data['appereances'] < 3].reset_index(drop=True)

In [85]:
not_frequent_universities

,university_name,appereances
0,University of Wisconsin,1
1,Medical University of South Carolina,2
2,University of Medicine and Dentistry of New Je...,1
3,University of Hamburg,1
4,University of Kentucky,2
...,...,...
403,Xidian University,1
404,Yeungnam University,1
405,Yıldız Technical University,1
406,Yokohama City University,1


**13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).**

Si effettua il merge tra i 3 dataset contenenti i rankings.

In [86]:
times_and_shanghai = times_data.merge(shanghai_data, on = ['university_name','year'], suffixes = ['_times','_shanghai'])

In [87]:
three_rankings = times_and_shanghai.merge(cwur_data, left_on = ['university_name', 'year'], right_on = ['institution', 'year'], suffixes = ['','_cwur'])

In [88]:
three_rankings.columns

Index(['world_rank_times', 'university_name', 'country', 'teaching',
       'international', 'research', 'citations', 'income', 'total_score_times',
       'num_students', 'student_staff_ratio', 'international_students',
       'female_male_ratio', 'year', 'world_rank_shanghai', 'national_rank',
       'total_score_shanghai', 'alumni', 'award', 'hici', 'ns', 'pub', 'pcp',
       'world_rank', 'institution', 'country_cwur', 'national_rank_cwur',
       'quality_of_education', 'alumni_employment', 'quality_of_faculty',
       'publications', 'influence', 'citations_cwur', 'broad_impact',
       'patents', 'score'],
      dtype='object')

Si estraggono solo le colonne di interesse, cioè quelle che contengono i rankings e l'anno per ciascuna università.

In [89]:
selected_three_rankings = three_rankings[['university_name','year','world_rank_times','world_rank_shanghai','world_rank']]

In [90]:
selected_three_rankings.dtypes

university_name        object
year                    int64
world_rank_times       object
world_rank_shanghai    object
world_rank              int64
dtype: object

La funzione *equal_rankings* prende in input una riga del dataframe. Innanzitutto converte il valore della colonna *world_rank* da intero a *stringa* e quindi valuta l'uguaglianza tra i 3 valori proposti per i 3 diversi rankings. La funzione resituisce un valore che è uguale a 1 se i tre valori sono uguali, 0 altrimenti.
NB: per i casi in cui il valore del ranking è espresso con una fascia di valori, l'uguaglianza non sarà mai rispettata se confrontata tale fascia con un singolo numero.

In [91]:
def equal_rankings(row):
    value = 0
    
    rank1 = row['world_rank_times']
    rank2 = row['world_rank_shanghai']
    rank3 = str(row['world_rank'])
    
    all_equal = (rank1 == rank2 == rank3)
    if all_equal:
        value = 1
    
    return value

Si assegnano i valori calcolati con la funzione *equal_rankings* a una nuova colonna e si estraggono le università che hanno un valore pari a 1 per tale colonna. Solo l'università di Stanford ha assunto la stessa posizione per tutti e tre i rankings e solo nell'anno 2013, posizionandosi seconda.

In [92]:
selected_three_rankings.loc[:,'equal_rankings'] = selected_three_rankings.apply(equal_rankings, axis = 1)

In [93]:
selected_three_rankings[selected_three_rankings['equal_rankings']>0].reset_index(drop=True)

,university_name,year,world_rank_times,world_rank_shanghai,world_rank,equal_rankings
0,Stanford University,2013,2,2,2,1
